In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/dv")

In [ ]:
data=pd.read_csv("zomato.csv",encoding = "ISO-8859-1")
print(data)

      Restaurant ID           Restaurant Name  Country Code              City  \
0           6317637          Le Petit Souffle           162       Makati City   
1           6304287          Izakaya Kikufuji           162       Makati City   
2           6300002    Heat - Edsa Shangri-La           162  Mandaluyong City   
3           6318506                      Ooma           162  Mandaluyong City   
4           6314302               Sambo Kojin           162  Mandaluyong City   
...             ...                       ...           ...               ...   
9546        5915730              NamlÛ± Gurme           208         ÛÁstanbul   
9547        5908749             Ceviz AÛôacÛ±           208         ÛÁstanbul   
9548        5915807                     Huqqa           208         ÛÁstanbul   
9549        5916112              Aôôk Kahve           208         ÛÁstanbul   
9550        5927402  Walter's Coffee Roastery           208         ÛÁstanbul   

                           

In [ ]:
data.duplicated().sum()
data.drop_duplicates(inplace=True)

In [ ]:
data.dropna(how='any',inplace=True)

In [ ]:
data.rename(columns={'Average Cost for two':'cost'},inplace=True)

In [ ]:
data['cost']=data['cost'].astype(str)
data['cost']=data['cost'].apply(lambda x: x.replace(',',''))
data['cost']=data['cost'].astype(float)

In [ ]:
restaurants = list(data['Restaurant Name'].unique())
data['Mean Rating'] = 0
for i in range(len(restaurants)):
    data['Mean Rating'][data['Restaurant Name'] == restaurants[i]] = data['Aggregate rating'][data['Restaurant Name'] == restaurants[i]].mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['Rating text'])

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(data['Restaurant Name'])

In [ ]:
def recommend(name, cosine_similarities = cosine_similarities):

    recommend_restaurant = []

    idx = indices[indices == name].index[0]

    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)

    top30_indexes = list(score_series.iloc[0:31].index)

    for each in top30_indexes:
        recommend_restaurant.append(list(data.index)[each])

    df_new = pd.DataFrame(columns=['Restaurant Name','Mean Rating', 'cost'])
    
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(data[['Restaurant Name','Mean Rating','cost']][data.index == each].sample()))
    
    df_new = df_new.drop_duplicates(subset=['Restaurant Name','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new

In [ ]:
recommend('Vikings')

TOP 10 RESTAURANTS LIKE Vikings WITH SIMILAR REVIEWS: 


,Restaurant Name,Mean Rating,cost
6618,Dukes Pastry Shop,4.200000,300.0
6620,London Street Kitchen,4.200000,600.0
6617,Dilli Treat,4.200000,500.0
6619,HotMess Bakes,4.100000,750.0
4956,QD's Restaurant,4.033333,800.0
9550,Walter's Coffee Roastery,4.000000,55.0
2609,Biryani Binge,3.900000,600.0
2610,City of Joy,3.900000,800.0
2629,Lemon Drops,3.900000,500.0
8179,Jungle Jamboree,3.825000,1200.0


In [ ]:
recommend('Ooma')

TOP 10 RESTAURANTS LIKE Ooma WITH SIMILAR REVIEWS: 


,Restaurant Name,Mean Rating,cost
9299,Milse,4.9,50.0
9303,Miann,4.9,25.0
304,Atlanta Highway Seafood Market,4.9,25.0
9296,Talaga Sampireun,4.9,200000.0
0,Le Petit Souffle,4.8,1100.0
2400,Spice Kraft,4.8,1200.0
9301,Depot Eatery and Oyster Bar,4.8,90.0
1839,Prankster,4.8,1500.0
352,Earl of Sandwich,4.7,35.0
9315,Eight - The Langham Hotel,4.7,190.0


In [ ]:
recommend('Earl of Sandwich')

TOP 10 RESTAURANTS LIKE Earl of Sandwich WITH SIMILAR REVIEWS: 


,Restaurant Name,Mean Rating,cost
9299,Milse,4.9,50.0
9303,Miann,4.9,25.0
304,Atlanta Highway Seafood Market,4.9,25.0
9296,Talaga Sampireun,4.9,200000.0
0,Le Petit Souffle,4.8,1100.0
2400,Spice Kraft,4.8,1200.0
9301,Depot Eatery and Oyster Bar,4.8,90.0
1839,Prankster,4.8,1500.0
352,Earl of Sandwich,4.7,35.0
9315,Eight - The Langham Hotel,4.7,190.0
